In [1]:
%pwd

'e:\\PROJECTS\\ML\\Brain-Tumor-Image-Classfication\\notebook'

In [2]:
%cd ..

e:\PROJECTS\ML\Brain-Tumor-Image-Classfication


c:\Users\reza\miniconda3\envs\brainMRI\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'e:\\PROJECTS\\ML\\Brain-Tumor-Image-Classfication'

In [19]:
from dataclasses import dataclass
import os
from brainMRI.logging import logger
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from pathlib import Path
import pickle

@dataclass
class Callbacks:
    root_dir: Path
    patience: int = 5
    factor: float = 0.1
    min_lr: float = 1e-6


    def get_callbacks(self) -> list:
        """
        A class that generates a set of callbacks for use in Keras model training.

        Attributes:
            log_dir (str): The directory to store the TensorBoard logs.
            checkpoint_dir (str): The directory to store the model checkpoint files.
            patience (int): The number of epochs with no improvement after which training will be stopped.
            factor (float): The factor by which the learning rate will be reduced on plateau.
            min_lr (float): The minimum learning rate.
    """
        
        checkpoint_dir = self.root_dir + '/ckpt'
        # Create the ModelCheckpoint callback
        model_checkpoint = ModelCheckpoint(
            os.path.join(checkpoint_dir, 'model-{epoch:02d}-{val_accuracy:.2f}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False
        )
        log_dir = self.root_dir + '/logs'
        # Create the TensorBoard callback
        tensorboard = TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )

        # Create the EarlyStopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=self.patience,
            verbose=1
        )

        # Create the ReduceLROnPlateau callback
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=self.factor,
            patience=self.patience // 2,
            min_lr=self.min_lr,
            verbose=1
        )

        logger.info("Callbacks initialized successfully.")

        callbacks = [model_checkpoint, tensorboard, early_stopping]
        callbacks_path = os.path.join(self.root_dir, "callbacks.pickle")
        with open(callbacks_path, 'wb') as handle:
            pickle.dump(callbacks, handle, protocol=pickle.HIGHEST_PROTOCOL)

        return [model_checkpoint, tensorboard, early_stopping, reduce_lr]


In [20]:
from brainMRI.constants import *
from brainMRI.utils.helpers import load_config, create_directories

class ConfigHandler:
    def __init__(self, file_path=CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = load_config(file_path)
        self.params = load_config(params_path)
        create_directories([self.config.root_dir])

    
    def get_callbacks_config(self) -> Callbacks:
        config = self.config.callbacks
        params = self.params.callbacks
        create_directories([config.root_dir])

        callbacks_config = Callbacks(
            root_dir=config.root_dir,
            patience=params.patience,
            factor=params.factor,
            min_lr=params.min_lr
        )
        return callbacks_config

In [21]:
try:
    config = ConfigHandler()
    callbacks_config = config.get_callbacks_config()
    f = callbacks_config.get_callbacks()
   
    print(f)
except Exception as e:
    raise e

[2024-05-18 09:56:56,373: INFO: helpers: YAML file: config\config.yaml loaded successfully]
[2024-05-18 09:56:56,384: INFO: helpers: YAML file: params.yaml loaded successfully]
[2024-05-18 09:56:56,386: INFO: helpers: Created directory at: project_outputs]
[2024-05-18 09:56:56,388: INFO: helpers: Created directory at: project_outputs/callbacks]
[2024-05-18 09:56:56,390: INFO: 671079601: Callbacks initialized successfully.]
[<keras.src.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000027E82681580>, <keras.src.callbacks.tensorboard.TensorBoard object at 0x0000027E835D3F20>, <keras.src.callbacks.early_stopping.EarlyStopping object at 0x0000027E835B4E60>, <keras.src.callbacks.reduce_lr_on_plateau.ReduceLROnPlateau object at 0x0000027E835B68A0>]
